# 모듈 import

In [ ]:
!pip install selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
pd.set_option('display.max_columns', None) # 데이터프레임 컬럼 display

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings('ignore') # 경고 무시

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 위의 과정을 함수화
- get_result
    - parameter : driver
    - return : set_result, date, date, team1, team2
- clean_result
    - parameter : set_result, date, date, team1, team2
    - return : team

In [ ]:
from re import L


def get_result(driver) :
    #--------------------- table 태그 읽기 - 팀 별 1 ~ 5 세트 결과 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    set_result = pd.read_html(str(table))[1] # 1 ~ 5 세트 별 득점 결과

    #--------------------- 날짜 정보 읽기 ---------------------#
    date_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > thead > tr > th'
    date = soup.select_one(date_selector).get_text()
    date = date.replace('\xa0', '').split('/')[0].strip().split()
    date = pd.to_datetime(date[0][:-1] + '/' + date[1][:-1] + '/' + date[2][:-1])

    ## by.가은
    #--------------------- 경기장 정보 읽기 ---------------------#
    stadium_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > tfoot > tr:nth-child(1) > td:nth-child(2) > span'
    stadium = soup.select_one(stadium_selector).get_text()
    stadium = stadium.split('/')[0].strip()

    #--------------------- 관중수 정보 읽기 ---------------------#
    crowd = soup.select_one(stadium_selector).get_text()
    crowd_str = list(crowd.split('/')[2].strip().split())[1][:-1]
    crowd = ''
    for c in crowd_str:
        if c != ',':
            crowd += c
    crowd = int(crowd) 
    #--------------------- 세트스코어 정보 읽기 ---------------------#
    set_score1_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > tbody > tr > td:nth-child(2) > p.num'
    set_score1 = soup.select_one(set_score1_selector).get_text().strip()
    set_score2_selector = '#wrp_content > article.wrp_recentgame.wrp_result > table > tbody > tr > td:nth-child(4) > p.num'
    set_score2 = soup.select_one(set_score2_selector).get_text().strip()



    # 첫 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 선수 기록 ---------------------#
    x_path = '//*[@id="wrp_content"]/article[2]/ul/li[2]/a/span' # 선수 기록 버튼
    cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team1 = pd.read_html(str(table))[3] # 명단 table
    team1_info = pd.read_html(str(table))[4] # 출전 세트, 득점, 공격 종합 table
    team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
    team1 = pd.concat([team1, team1_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team1"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team1_info = pd.read_html(str(table))[4] # 다음 기록
        team1_info.columns = ['_'.join(col).strip() for col in team1_info.columns]
        team1 = pd.concat([team1, team1_info], axis = 1)


    #
    # 두 번째 팀 정보 읽기
    #

    #--------------------- 버튼 클릭 - 두 번째 팀 ---------------------#
    try :
        x_path = '//*[@id="tab2"]/div[3]/ul/li[2]/a/span' # 두 번째 팀 버튼
        cursor = driver.find_element(By.XPATH,x_path)
    except :
        try :
            x_path = '//*[@id="tab2"]/div[2]/ul/li[2]/a/span' # 두 번째 팀 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            x_path = '//*[@id="tab2"]/div[1]/ul/li[2]/a/span' # 두 번째 팀 버튼
            cursor = driver.find_element(By.XPATH,x_path)
    cursor.click()
    time.sleep(1)

    #--------------------- table 태그 읽기 - 선수 별 출전 세트, 득점, 공격 종합 ---------------------#
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')

    team2 = pd.read_html(str(table))[5] # 명단 table
    team2_info = pd.read_html(str(table))[6] # 출전 세트, 득점, 공격 종합 table
    team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
    team2 = pd.concat([team2, team2_info], axis = 1)

    #--------------------- 나머지 기록 읽기 ---------------------#
    for _ in range(4) :
        #--------------------- 버튼 클릭 - 다음 기록 ---------------------#
        x_path = '//*[@id="team2"]/div/div[1]/a[2]' # 다음 기록 버튼
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)


        #--------------------- table 태그 읽기 - 다음 기록들 ---------------------#
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table = soup.find_all('table')

        team2_info = pd.read_html(str(table))[6] # 다음 기록
        team2_info.columns = ['_'.join(col).strip() for col in team2_info.columns]
        team2 = pd.concat([team2, team2_info], axis = 1)
    
    return set_result, date, team1, team2, stadium, crowd, set_score1, set_score2

In [ ]:
def clean_result(set_result, date, team1, team2, stadium, crowd, set_score1, set_score2) :
    # No. 열 삭제
    team1 = team1.drop('No.', axis = 1)
    team2 = team2.drop('No.', axis = 1)

    
    # 3경기 이상 출전한 선수만 유지 (출전 세트 결측치가 3개 이상인 선수 제거)
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    # thresh = 3 : 정상 값(Not NaN)이 3개 이상만 유지
    team1 = team1.dropna(subset = subset, thresh = 3)
    team2 = team2.dropna(subset = subset, thresh = 3)

    # 스타팅멤버 표시 (3세트 이상 스타팅멤버라면 1 아니면 0)
    team1['스타팅멤버'] = 0
    team2['스타팅멤버'] = 0

    team1.loc[(team1.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    team2.loc[(team2.iloc[:, 1:6] == 'O').sum(axis = 1) >= 3, '스타팅멤버'] = 1
    
    
    # 출전 세트 열 삭제
    subset = ['출전세트_1set', '출전세트_2set', '출전세트_3set', '출전세트_4set', '출전세트_5set']

    team1 = team1.drop(subset, axis = 1)
    team2 = team2.drop(subset, axis = 1)

    
    # 이름과 포지션 분리하기
    # "이름" 열 : 이름 정보
    # "포지션" 열 : 포지션 정보
    team1['포지션'] = team1['이름'].str.split().str[1].str.strip('()')
    team1['이름'] = team1['이름'].str.split().str[0]

    team2['포지션'] = team2['이름'].str.split().str[1].str.strip('()')
    team2['이름'] = team2['이름'].str.split().str[0]

    
    # 선수 별 득점 점유율 구하기 (개인 별 득점을 팀 전체의 득점으로 나누기)
    team1_total_score = team1['득점_득점'].sum()
    team2_total_score = team2['득점_득점'].sum()
    team1['득점점유율'] = team1['득점_득점'] / team1_total_score
    team2['득점점유율'] = team2['득점_득점'] / team2_total_score
    
    
    # 컬럼 선택
    columns = [
        '이름',
        '득점_득점',
        '공격종합_시도',
        '공격종합_성공',
        '공격종합_공격차단',
        '공격종합_범실',
        '공격종합_성공률',
        '공격종합_점유율',
        '오픈_시도',
        '오픈_성공',
        '오픈_공격차단',
        '오픈_범실',
        '오픈_성공률',
        '오픈_점유율',
        '시간차_시도',
        '시간차_성공',
        '시간차_공격차단',
        '시간차_범실',
        '시간차_성공률',
        '시간차_점유율',
        '이동_시도',
        '이동_성공',
        '이동_공격차단',
        '이동_범실',
        '이동_성공률',
        '이동_점유율',
        '후위_시도',
        '후위_성공',
        '후위_공격차단',
        '후위_범실',
        '후위_성공률',
        '후위_점유율',
        '속공_시도',
        '속공_성공',
        '속공_공격차단',
        '속공_범실',
        '속공_성공률',
        '속공_점유율',
        '퀵오픈_시도',
        '퀵오픈_성공',
        '퀵오픈_공격차단',
        '퀵오픈_범실',
        '퀵오픈_성공률',
        '퀵오픈_점유율',
        '서브_시도',
        '서브_성공',
        '서브_범실',
        '서브_성공률',
        '서브_점유율',
        '디그_시도',
        '디그_성공',
        '디그_실패',
        '디그_범실',
        '디그_세트당',
        '디그_점유율',
        '세트_시도',
        '세트_성공',
        '세트_범실',
        '세트_세트당',
        '세트_점유율',
        '리시브_시도',
        '리시브_정확',
        '리시브_실패',
        '리시브_세트당',
        '리시브_점유율',
        '블로킹_시도',
        '블로킹_성공',
        '블로킹_유효블락',
        '블로킹_실패',
        '블로킹_범실',
        '블로킹_세트당',
        '블로킹_점유율',
        '블로킹_어시스트',
        '벌칙_벌칙',
        '범실_범실',
        '포지션',
        '득점점유율',
        '스타팅멤버'
    ]
    team1 = team1[columns]
    team2 = team2[columns]
    
    
    # 경기날짜, 팀명, 결과 정보 추가
    team1['팀명'] = set_result.iloc[0,0]
    team2['팀명'] = set_result.iloc[1,0]
    team1['결과'] = 1 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 0
    team2['결과'] = 0 if int(set_result.iloc[0,-1]) > int(set_result.iloc[1,-1]) else 1
    team1['경기날짜'] = date
    team2['경기날짜'] = date
    team1['경기장'] = stadium
    team2['경기장'] = stadium
    team1['관중수'] = crowd
    team2['관중수'] = crowd
    team1['1팀score'] = set_score1
    team1['2팀score'] = set_score2
    team2['1팀score'] = set_score1
    team2['2팀score'] = set_score2

    # 컬럼 순서 변경
    team1 = team1[['팀명', '경기장', '관중수', '경기날짜', '1팀score', '2팀score', '결과'] + columns]
    team2 = team2[['팀명', '경기장', '관중수', '경기날짜', '1팀score', '2팀score', '결과'] + columns]
    
    
    # 연결
    team = pd.concat([team1, team2])
    team
    
    return team

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

# 모든 경기의 정보 가져오기
- 경기 스케쥴 url 확인
- 여자 배구 상세결과 url 확인
- 각 url 별 함수 적용
- 파일로 저장

#### 경기 스케쥴 url 확인
- 도드람 2017-2018 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2018-03&r_round=
    
- 도드람 2018-2019 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2018-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=015&team=&yymm=2019-03&r_round=
    
- 도드람 2018-2020 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=016&team=&yymm=2019-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=016&team=&yymm=2020-03&r_round=

- 도드람 2020-2021 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=017&team=&yymm=2020-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=017&team=&yymm=2021-04&r_round=

- 도드람 2021-2022 V-리그 url
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=018&team=&yymm=2021-10&r_round=
    - ...
    - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=018&team=&yymm=2022-02&r_round=

In [ ]:
# 반복문으로 각 url 출력
seasons = [f'0{num}' for num in range(14, 19)]
yymms = [
pd.date_range('2017-10', '2018-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2018-10', '2019-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2019-10', '2020-04', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2020-10', '2021-05', freq = 'M').strftime('%Y-%m').to_list(),
pd.date_range('2021-10', '2022-03', freq = 'M').strftime('%Y-%m').to_list()
]
# print(seasons)
# print(yymm)

for season, dates in zip(seasons, yymms) :
    for date in dates :
        url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
        print(url)

<br/>
<br/>
<br/>
<br/>

#### 여자 배구 상세결과 url 확인
- 남자 / 여자 XPath 확인
- 상세결과 버튼 XPath 확인
- 경기가 없는 날의 XPath 확인

In [ ]:
# url - https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round=
'''
//*[@id="type1"]/div/table/tbody/tr[1]/td[3] # 남자
//*[@id="type1"]/div/table/tbody/tr[1]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[2]/td[3] # 여자
//*[@id="type1"]/div/table/tbody/tr[2]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[3]/td[3] # 남자
//*[@id="type1"]/div/table/tbody/tr[3]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[4]/td[3] # 여자
//*[@id="type1"]/div/table/tbody/tr[4]/td[10]/a[2] # 상세결과


//*[@id="type1"]/div/table/tbody/tr[5]/td[3] # 경기가 없습니다.


//*[@id="type1"]/div/table/tbody/tr[6]/td[3] # 여자
//*[@id="type1"]/div/table/tbody/tr[6]/td[10]/a[2] # 상세결과
...


'''

In [ ]:
# !------------ 이 코드 필요 없는 것 같긴 한데!! --------------! #

# ## 여자 경기의 상세결과 접근하기

# # 경기 스케줄 url 접속
# driver = webdriver.Chrome()
# url = "https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season=014&team=&yymm=2017-10&r_round="
# driver.get(url)
# time.sleep(2)


# # 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
# html = driver.page_source
# soup = BeautifulSoup(html, 'html.parser')
# table = soup.find_all('table')
# df = pd.read_html(str(table))[0]
# idx = df[df['구분'] == '여자'].index.to_list()
# idx = [i + 1 for i in idx]

# dfs = []

# # 각 경기의 상세결과 버튼 클릭
# for i in idx :
#     # 상세결과 버튼 클릭
#     try :
#         x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
#         cursor = driver.find_element(By.XPATH,x_path)
#     except :
#         x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
#         cursor = driver.find_element(By.XPATH,x_path)
#     cursor.click()
#     time.sleep(1)
    
#     # 함수 적용
#     set_result, date, team1, team2 = get_result(driver)
#     team = clean_result(set_result, date, team1, team2)
#     dfs.append(team)
    
#     # 다시 경기 스케줄 url로 이동
#     driver.get(url)
#     time.sleep(1)
# df = pd.concat(dfs)
# df

<br/>
<br/>
<br/>
<br/>

- 수동 실행 (시즌 별)

In [ ]:
dfs = []
# 이 seasons의 index를 수정해서 돌려주면 됩니다! 인덱스 별 진행상황은 카톡을 확인해주세요!!!!!!!!
season = seasons[4]
dates = yymms[4]
driver = webdriver.Chrome()

for date in dates :
    # 경기 스케줄 url 접속
    url = f'https://www.kovo.co.kr/game/v-league/11110_schedule_list.asp?season={season}&team=&yymm={date}&r_round='
    driver.get(url)
    time.sleep(1)


    # 경기 스케줄을 데이터프레임으로 저장 후 여자 경기의 인덱스를 idx에 저장
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find_all('table')
    try :
        df = pd.read_html(str(table))[0]
        idx = df[df['구분'] == '여자'].index.to_list()
    except :
        df = pd.read_html(str(table))[2]
        idx = df[df['구분'] == '여자'].index.to_list()
    idx = [i + 1 for i in idx]



    # 각 경기의 상세결과 버튼 클릭
    for i in idx :
        # 상세결과 버튼 클릭
        try :
            x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[3]' # 상세결과 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        except :
            x_path = f'//*[@id="type1"]/div/table/tbody/tr[{i}]/td[10]/a[2]' # 상세결과 버튼
            cursor = driver.find_element(By.XPATH,x_path)
        cursor = driver.find_element(By.XPATH,x_path)
        cursor.click()
        time.sleep(1)

        # 함수 적용
        set_result, date, team1, team2, stadium, crowd, set_score1, set_score2 = get_result(driver)
        team = clean_result(set_result, date, team1, team2, stadium, crowd, set_score1, set_score2)
        dfs.append(team)

        # 다시 경기 스케줄 url로 이동
        driver.get(url)
        time.sleep(1)
        
# csv 파일_fin로 저장
df = pd.concat(dfs)
df.to_csv(f'volleyball_{season}_fin.csv', index = False)
df = pd.read_csv(f'volleyball_{season}_fin.csv')
df

<br/>
<br/>
<br/>
<br/>

#### 데이터 연결 (각자 실행 후 최종 concat 진행하겠습니다.)

In [ ]:
# dfs = []
# for season in seasons :
#     dfs.append(pd.read_csv(f'volleyball_{season}.csv'))
# df = pd.concat(dfs)
# df.to_csv(f'volleyball_{season}.csv', index = False)
# df = pd.read_csv(f'volleyball_{season}.csv')
# df

In [ ]:
# # 경기날짜의 데이터타입 변경 (datetime64)
# df['경기날짜'] = df['경기날짜'].astype('datetime64')
# df.dtypes

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>